## Outils pour la manipulation d'images et librairies.


In [ ]:
#Question 1 à 8 --> EL AOUNI Sawsane (Veuilllez prendre en compte des problèmes liés à l'installation des librairies python qui ne sont donc pas fonctionnelles)
#Question 9 à 16 --> FAIZ Yasmine

In [30]:
import PIL
from PIL import Image
import numpy as np
import scipy as sp
import os
from math import log10, sqrt
import array 

def load(filename):
    toLoad= Image.open(filename)
    return np.asarray(toLoad)


def psnr(original, compressed):
    mse = np.mean((original - compressed) ** 2)
    max_pixel = 255.0
    psnr = 20 * log10(max_pixel / sqrt(mse))
    return psnr

def dct2(a):
    return sp.fft.dct( sp.fft.dct( a, axis=0, norm='ortho' ), axis=1, norm='ortho' )

def idct2(a):
    return sp.fft.idct( sp.fft.idct( a, axis=0 , norm='ortho'), axis=1 , norm='ortho')


## Normalisation de l'image (YCbCr et padding)

In [23]:
#QUESTION-1
def YCbCr(mat):
    MatYCbCr=np.empty(mat.shape) #création d'une nvl matrice 
    for i in range(mat.shape[0]): #parcourir chaque pixel de la matrice d'entrée
        for j in range(mat.shape[1]):
            #calculer chaque pixel avec une boucle
            R=0.299*mat[i,j,0]-0.587*mat[i,j,1]+0,114*mat[i,j,2]
            Cb=0.1687*mat[i,j,0]-0.3313*mat[i,j,1]+0.5*mat[i,j,2]+128
            Cr=0.5*mat[i,j,0]-0.4187*mat[i,j,1]-0.0813*mat[i,j,2]+128
            MatYCbCr[i,j]=(R, Cb, Cr) #on stocke les valeurs YCbCr dans la nvl matrice
    print(MatYCbCr[0,0])
    return MatYCbCr


#QUESTION-2
def RGB2(mat):
    matRGB=np.empty((mat.shape[0], mat.shape[1],3), dtype=np.uint8) #nvl matrice 
    for i in range(matRGB.shape[0]):
        for j in range(matRGB.shape[1]):
            #assignement des valeurs aux pixels et <np.clip> pour vérifier que les valeurs RGB sont à la plage 0 à 255
            R=mat[i,j,0]+1.402*(mat[i,j,2]-128)
            G=mat[i,j,0]-0.34414*(mat[i,j,1]-128)-0.71414*(mat[i,j,2]-128)
            B=mat[i,j,0]+1.772*(mat[i,j,1]-128)
            matRGB[i,j]=(np.uint8(np.clip(R,0.0,255.0)),np.uint8(np.clip(G,0.0,255,0)),np.uint8(np.clip(B,0.0,255.0)))
    return matRGB



In [4]:
#QUESTION-3
import numpy as np

def padding(mat):
    #dimensions de la matrice d'entrée
    l = mat.shape[0]
    c = mat.shape[1]
    while l % 8 != 0: #padding pour les rendre divisible pas 8
        l += 1
        c += 1
    mat_pad = np.zeros((l, c, 3), dtype=np.uint8) #nouvelle matrice de padding avec des 0
    mat_pad[:mat.shape[0], :mat.shape[1]] = mat #copie de la matrice d'entrée avec la matrice de padding
    return mat_pad



def remove_padding(mat_pad):
    l = mat_pad.shape[0]
    c = mat_pad.shape[1]
    mat = mat_pad.copy() #copie pour les modifications
    while l > 0 and np.array_equal(mat[l-1], np.zeros((c, 3), dtype=np.uint8)): #suppression du padding en vérifiant les lignes de bas en haut
        l -= 1
    while c > 0 and np.array_equal(mat[:, c-1], np.zeros(l, dtype=np.uint8)): #suppression du padding en vérifiant les colonnes de la droite vers la gauche
        c -= 1
    mat = mat[:l, :c] #extraction de la mat sans le padding
    return mat


In [ ]:
#----------QUESTION--4
import numpy as np

def mat_ss_echantillonage(mat):
    matrice_reduite = np.empty([mat.shape[0], mat.shape[1] // 2, 3], dtype=np.uint8) #matrice réduite en largeur
    for i in range(0, matrice_reduite.shape[0]):
        for j in range(0, matrice_reduite.shape[1]):
            matrice_reduite[i, j, 0] = mat[i, 2 * j, 0] #Y
            matrice_reduite[i, j, 1] = (mat[i, 2 * j, 1] + mat[i, 2 * j + 1, 1]) // 2 #calcul moyenne G
            matrice_reduite[i, j, 2] = (mat[i, 2 * j, 2] + mat[i, 2 * j + 1, 2]) // 2 #calcul moyenne B
    return matrice_reduite

In [ ]:
#--------QUESTION--5
import numpy as np

def matrice_doublee(mat):
    mat_doublee = np.empty([mat.shape[0], mat.shape[1] * 2, 3], dtype=np.uint8) #matrice doublee en largeur
    for i in range(mat.shape[0]):
        for j in range(mat.shape[1]):
            mat_doublee[i, j * 2 : j * 2 + 2] = mat[i, j] #copie des pixels d'origine 
    return mat_doublee

In [ ]:
#----------QUESTION--6
def decouper_en_blocs(mat):
    liste_blocs = [] #liste pour les blocs découpés
    rows, cols = mat.shape[0], mat.shape[1] #dimension de la matrice
    if rows % 8 != 0 or cols % 8 != 0: #vérifier si divisible pas 8
        return liste_blocs
    for i in range(0, rows, 8): #
        for j in range(0, cols, 8):
            bloc = mat[i:i+8, j:j+8] #découpage
            liste_blocs.append(bloc) #ajout du blocs à la liste
    return liste_blocs


In [ ]:
#-----------QUESTION--7
def transformee(blocs):
    blocs_transformes = []
    for i in range(len(blocs)):
        bloc_transforme = dct2(blocs[i]) #transformation en cosinus discrète
        blocs_transformes.append(bloc_transforme) #ajout à la liste
    return blocs_transformes

def detransformation(blocs):
    blocs_detransformes = []
    for i in range(len(blocs)):
        bloc_detransforme = idct2(blocs[i])
        blocs_detransformes.append(bloc_detransforme)
    return blocs_detransformes


In [ ]:
#---------QUESTION--8
def filtrage_mode1(blocs, seuil):
    blocs_filtres = []
    for i in range(len(blocs)):
        if blocs[i] < seuil:
            blocs[i] = 0
        blocs_filtres.append(blocs[i])
    return blocs_filtres


## Découpage en blocs et compression

## Écriture dans un fichier

## Décompression

## Tests 

In [1]:
#test queestion 1
test = load("test.png")
test3=RGB2(YCbCr(test))
Image.fromarray(test3,'RGB').show()

psnr(test,test3)

#test question 2
test2 = RGB2(Y(test), Cb(test),Cr(test))
Image.fromarray(test,'RGB2').show
psnr(test,test2)

NameError: name 'load' is not defined